<a href="https://colab.research.google.com/github/NattachaiWat/ml442/blob/batch4/Lab1_2_opencv_dnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
mkdir model
mkdir images
cd model
wget -q https://storage.googleapis.com/nattachai-ai-data/share/model/age_net.caffemodel -O age_net.caffemodel
wget -q https://storage.googleapis.com/nattachai-ai-data/share/model/gender_net.caffemodel -O gender_net.caffemodel
wget -q https://storage.googleapis.com/nattachai-ai-data/share/model/deploy_age.prototxt -O deploy_age.prototxt
wget -q https://storage.googleapis.com/nattachai-ai-data/share/model/deploy_gender.prototxt -O deploy_gender.prototxt 
cd ../images
wget -q https://storage.googleapis.com/nattachai-ai-data/share/images/mypic-001.png -O mypic-001.png 
wget -q https://storage.googleapis.com/nattachai-ai-data/share/images/sample.png -O sample.png
cd ..

In [ ]:
import os
import numpy as np
import time
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
age_list=['age (0-2)','age (4-6)','age (8-12)','age (15-20)','age (25-32)','age (38-43)','age (48-53)','age (60-100)']
gender_list=['Male','Female']

In [ ]:
age_model_prototxt = './model/deploy_age.prototxt'
age_model_weight   = './model/age_net.caffemodel'
age_model_shape    = [1, 3, 227, 227]

gender_model_prototxt = './model/deploy_gender.prototxt'
gender_model_weight   = './model/gender_net.caffemodel'
gender_model_shape    = [1, 3, 227, 227]

In [ ]:
!ls images

In [ ]:
#@title Set image filename
filename = "" #@param {type:"string"}

image_filename  = os.path.join('images',filename)

face = cv2.imread(image_filename)

# 
faceBlob = cv2.dnn.blobFromImage(face, 1.0, (227, 227),
			(78.4263377603, 87.7689143744, 114.895847746),
			swapRB=False)

In [ ]:
#@title Show image { vertical-output: true }
cv2_imshow(face)

In [ ]:
#@title Define function: must run it { vertical-output: true }
def load_and_show_model(model_prototxt,model_weight,image_shape):
    print("Loading model...")
    net = cv2.dnn.readNetFromCaffe(model_prototxt, model_weight)

    if not net.empty():
        print('Net loaded successfully\n')
        
    print('Net contains:')
    for t in net.getLayerTypes():
        print('{} layers of type {}'.format(net.getLayersCount(t), t))

    # image size 
    layers_ids, in_shapes, out_shapes = net.getLayersShapes(image_shape)

    layers_names = net.getLayerNames()

    print('Net layers shapes:')
    for l in range(len(layers_names)):
        in_num, out_num = len(in_shapes[l]), len(out_shapes[l])
        print('Layer {} has {} input(s) and {} output(s)'.format(layers_names[l], in_num, out_num))
        for i in range(in_num):
            print('\tinput #{} has shape'.format(i, in_shapes[l][i].flatten()))
        
        for i in range(out_num):
            print('\toutput #{} has shape'.format(i, out_shapes[l][i].flatten()))
    return net
def predict(image_blob,net):
    net.setInput(image_blob)
    prob = net.forward()
    return prob

In [ ]:
#@title Load model { vertical-output: true }
age_net = load_and_show_model(age_model_prototxt,age_model_weight,age_model_shape)
gender_net = load_and_show_model(gender_model_prototxt,gender_model_weight,gender_model_shape)


In [ ]:
#@title Show output { vertical-output: true }
print(age_list[age], gender_list[gender])